In [1]:
import yaml
from box import ConfigBox
from box.exceptions import BoxValueError
from pathlib import Path
import os
from pydantic import BaseModel
import urllib.request as request
from utils import *
from utils import read_custom_csv
import requests
import zipfile
import re
import pandas as pd


In [4]:
data_root_path = os.path.join('artifacts', 'data')
files = os.listdir(data_root_path)
print(files)
train_data_path = os.path.join(data_root_path, 'adult.data')
test_data_path =  os.path.join(data_root_path, 'adult.test')
text_file_path =  os.path.join(data_root_path, 'adult.names')

with open(text_file_path) as text:
    cols = []
    for line in text:
        sre = re.match(r'(?P<colname>[a-z\-]+):.*\.', line)
        if sre:
            cols.append(sre.group('colname'))
    cols.append('label')
    

['adult.data', 'adult.names', 'adult.test', 'Index', 'old.adult.names']


In [5]:
options = {'header': None, 'names': cols, 'skipinitialspace': True}
income_df = pd.read_csv(train_data_path, **options)
income_df.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'label'],
      dtype='object')

In [10]:
class PathValidator(BaseModel):
    root_dir: Path
    unzipped_dir: Path
    STATUS_FILE: Path
    schema_dir: Path

def validate_data(configbox):
    config = configbox.data_validation
    config = PathValidator(**config)
    return config

In [22]:
class DataValidation:
    def __init__(self, config):
        self.config = config

    def validate_columns(self):
        create_directories([self.config.root_dir])
        validation_status = None
        data = read_custom_csv(self.config.unzipped_dir)
        all_columns = list(data.columns)
        schema_config = read_yaml_file(Path(self.config.schema_dir))
        schema = schema_config.COLUMNS.keys()
        for column in all_columns:
            if column not in schema:
                validation_status = False
                with open(self.config.STATUS_FILE, 'w') as file:
                    file.write(f"validation status: {validation_status}")
                
            else:
                validation_status = True
                with open(self.config.STATUS_FILE, 'w') as file:
                    file.write(f"validation status: {validation_status}")
        return validation_status  

In [25]:
config_box = read_yaml_file("artifacts/ConfigFiles/config.yaml")
config = validate_data(config_box)
data_validation = DataValidation(config)
data_validation.validate_columns()


yaml file: artifacts/ConfigFiles/config.yaml loaded successfully
created directory at artifacts\data_validation
yaml file: artifacts\ConfigFiles\schema.yaml loaded successfully


True